In [1]:
import pandas as pd
import numpy as np

In [65]:
ratings = pd.read_csv('u.data', 
                      sep='\t', 
                      encoding='latin-1',header = None)
                      #usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings[0].drop_duplicates().max()
max_movieid = ratings[1].drop_duplicates().max()
                     
print(len(ratings), 'ratings')
max_userid

100000 ratings


943

In [28]:
K_FACTORS = 120
RNG_SEED = 1446557
ratings.head()


,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [71]:
shuffled_ratings = ratings.sample(frac=1., random_state=RNG_SEED)
Users = shuffled_ratings[0].values-1
print('Users:', Users, ', shape =', Users.shape)
Movies = shuffled_ratings[1].values-1
print ('Movies:', Movies, ', shape =', Movies.shape)
Ratings = shuffled_ratings[2].values
print('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [304 902 928 ..., 550 654 315] , shape = (100000,)
Movies: [ 732 1069  134 ...,  229  274   88] , shape = (100000,)
Ratings: [3 4 5 ..., 5 4 1] , shape = (100000,)


In [120]:
print(ratings[(ratings[0] == 305) & (ratings[1] == 733)])



Users

        0    1  2          3
5064  305  733  3  886324661


array([304, 902, 928, ..., 550, 654, 315], dtype=int64)

In [73]:
import numpy as np
from keras.layers import Embedding, Reshape, Merge, Dropout, Dense
from keras.models import Sequential

class CFModel(Sequential):

    def __init__(self, n_users, m_items, k_factors, **kwargs):
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))
        super(CFModel, self).__init__(**kwargs)
        self.add(Merge([P, Q], mode='dot', dot_axes=1))

    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]

In [74]:
model = CFModel(max_userid, max_movieid, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')

C:\vishal\anaconda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [112]:
import matplotlib.pyplot as plt
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint('new.h5', save_best_only=True)]
history = model.fit([Users, Movies], Ratings, nb_epoch=50, validation_split=.1, verbose=2, callbacks=callbacks)

C:\vishal\anaconda\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 90000 samples, validate on 10000 samples
Epoch 1/50
14s - loss: 0.4276 - val_loss: 0.8559
Epoch 2/50
13s - loss: 0.4041 - val_loss: 0.8594
Epoch 3/50
13s - loss: 0.3823 - val_loss: 0.8631
Epoch 4/50
13s - loss: 0.3607 - val_loss: 0.8672


In [113]:
users = pd.read_csv('u.user', sep='|', encoding='latin-1', header = None)
print(len(users), 'descriptions of', max_userid, 'users loaded.')

943 descriptions of 943 users loaded.


In [114]:
movies = pd.read_csv('u.item', sep='|', encoding='latin-1', header=None)
print(len(movies), 'descriptions of', max_movieid, 'movies loaded.')

1682 descriptions of 1682 movies loaded.


In [115]:
movies.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [116]:
users.head()

,0,1,2,3,4
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [117]:
test = 100
users[users[0] == test]


,0,1,2,3,4
99,100,36,M,executive,90254


In [118]:
def predict_rating(userid, movieid):
    return model.rate(userid , movieid)

In [119]:
user_ratings = ratings[ratings[0] == test][[0, 1, 2]]
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(test, x[2]), axis=1)
user_ratings

,0,1,2,prediction
270,100,344,4,2.761813
1008,100,354,2,2.451372
1434,100,268,3,2.778884
2411,100,321,1,2.852464
3175,100,355,4,2.761813
4262,100,750,4,2.761813
4295,100,266,2,2.451372
4711,100,288,2,2.451372
5142,100,302,4,2.761813
5540,100,340,3,2.778884
